__Author: Manu Jayadharan, University of Pittsburgh, 2020__

# Using fluidlearn to solve 3D poisson equation

Equation to solve: $-\Delta u -f  = 0$
over domain $\Omega$ .

For demonstration purposes we take $f=6(x_1 + x_2) + 2$ and $\Omega = [0,3]\times [-1,1]\times [-5,5]$ , so we can compare the results with the exact solution $u= x^3 + y^3 + z^2$.

In [3]:
#Import fluidlearn package and classes
import "../../../fluidlearn/fluidlearn"
from fluidlearn import dataprocess

SyntaxError: invalid syntax (<ipython-input-3-e54538b42d13>, line 2)

### Defining the domain and time interval for which the PDE needs to be solved.
This matters only for generating collocation points and if the user is feeding their own collocation points,
they can skip this step.

In [3]:
#domain range
X_1_domain = [-2, 2]
X_2_domain = [0, 1]
#time range
T_initial = 0
T_final = 1
T_domain = [T_initial, T_final]

#domain of the problem
domain_bounds = [X_1_domain, X_2_domain, T_domain]

### Loading data from a csv file
- We use the manufactured data with $u=tsin(x_1 + x_2)$ saved in a csv file.
- Data is saved in the format: ($x_1 , x_2, t, u(x_1, x_2, t)$) as four columns.
- You could load either preprocess your data to be in this format or load your data
  from a csv file with similar format.

In [4]:
path_to_data = "data_manufactured/t_sin_x_plus_y.csv"
X_data, Y_data = dataprocess.imp_from_csv(path_to_csv_file=path_to_data,
                                           x_y_combined=True, y_dim=1)

### Defining the rhs function $f=sin(x_1 + x_2) + tsin(x_1 + x_2)$ of the PDE.
We use tensorflow.sin function instead of python functions, we could used numpy.sin as well.

In [5]:
def rhs_function (args, time_dep=True):
        import tensorflow as tf
        if time_dep:
            space_inputs = args[:-1]
            time_inputs = args[-1]
        else:
            space_inputs = args
        
        return tf.sin(space_inputs[0]+space_inputs[1]) + 2*time_inputs*tf.sin(space_inputs[0]+space_inputs[1])

### Defining the model architecture

In [6]:
model_type = 'forward'
space_dim = 2 #dimension of Omega
time_depedent_problem = True
n_hid_lay=3 #numberof hidden layers in the neural network
n_hid_nrn=20 #number of neurons in each hidden layer
act_func='tanh' #activation function used for hidden layers:  could be elu, relu, sigmoid
loss_list='mse' #type of error function used for cost functin, we use mean squared error.
optimizer='adam' #type of optimizer for cost function minimization
dom_bounds=domain_bounds #domain bounds where collocation points has to be generated

distribution = 'uniform' #type of distribution used for generating the pde collocation points.
number_of_collocation_points = 5000

batch_size = 32 #batch size for stochastic batch gradient type optimization
num_epochs = 10 #number of epochs used for trainng  

### Defining the fluidlearn solver 

In [7]:
diffusion_model = fluidlearn.Solver()

In [8]:
diffusion_model(model_type=model_type,
            space_dim=space_dim,
            time_dep=time_depedent_problem,
            output_dim=1,
            n_hid_lay=n_hid_lay,
            n_hid_nrn=n_hid_lay,
            act_func=act_func,
            rhs_func=rhs_function,
            loss_list=loss_list,
            optimizer=optimizer,
            dom_bounds=dom_bounds,
            load_model=False,
            model_path=None,)

### Fitting the model

In [9]:
diffusion_model.fit(
    x=X_data,
    y=Y_data,
    colloc_points=number_of_collocation_points,
    dist=distribution,
    batch_size=batch_size,
    epochs=num_epochs,
)

Epoch 1/10
219/219 [==============================] - 0s 1ms/step - loss: 0.3513 - output_1_loss: 0.0209 - output_2_loss: 0.3305
Epoch 2/10
219/219 [==============================] - 0s 1ms/step - loss: 0.1598 - output_1_loss: 0.0228 - output_2_loss: 0.1370
Epoch 3/10
219/219 [==============================] - 0s 1ms/step - loss: 0.1110 - output_1_loss: 0.0217 - output_2_loss: 0.0893
Epoch 4/10
219/219 [==============================] - 0s 1ms/step - loss: 0.0940 - output_1_loss: 0.0224 - output_2_loss: 0.0717
Epoch 5/10
219/219 [==============================] - 0s 1ms/step - loss: 0.0831 - output_1_loss: 0.0231 - output_2_loss: 0.0600
Epoch 6/10
219/219 [==============================] - 0s 1ms/step - loss: 0.0732 - output_1_loss: 0.0231 - output_2_loss: 0.0501
Epoch 7/10
219/219 [==============================] - 0s 1ms/step - loss: 0.0628 - output_1_loss: 0.0214 - output_2_loss: 0.0414
Epoch 8/10
219/219 [==============================] - 0s 1ms/step - loss: 0.0514 - output_1_loss:

### Resuming Training  the model again for 50 more epochs

In [17]:
diffusion_model.fit(
    x=X_data,
    y=Y_data,
    colloc_points=number_of_collocation_points,
    dist=distribution,
    batch_size=batch_size,
    epochs=50,
)

Epoch 1/50
219/219 [==============================] - 0s 1ms/step - loss: 0.0010 - output_1_loss: 5.5980e-04 - output_2_loss: 4.7206e-04
Epoch 2/50
219/219 [==============================] - 0s 1ms/step - loss: 0.0010 - output_1_loss: 5.6017e-04 - output_2_loss: 4.6657e-04
Epoch 3/50
219/219 [==============================] - 0s 1ms/step - loss: 0.0010 - output_1_loss: 5.5690e-04 - output_2_loss: 4.6706e-04
Epoch 4/50
219/219 [==============================] - 0s 1ms/step - loss: 0.0010 - output_1_loss: 5.6190e-04 - output_2_loss: 4.6779e-04
Epoch 5/50
219/219 [==============================] - 0s 1ms/step - loss: 9.9724e-04 - output_1_loss: 5.5509e-04 - output_2_loss: 4.4215e-04
Epoch 6/50
219/219 [==============================] - 0s 1ms/step - loss: 9.8571e-04 - output_1_loss: 5.5845e-04 - output_2_loss: 4.2726e-04
Epoch 7/50
219/219 [==============================] - 0s 1ms/step - loss: 9.9226e-04 - output_1_loss: 5.5868e-04 - output_2_loss: 4.3358e-04
Epoch 8/50
219/219 [=========

### Demo Using the trained model for predicton

In [32]:
#taking two points from the domain for time t=0.3 and t=0.76 respectively
x_test_points = [[-0.5,0.1,0.3],
                [0.66,0.6,0.76]]
#Predicting the value
y_predicted = diffusion_model.predict(x_test_points)

In [33]:
#finding the true y value for comparing
import numpy as np
x_test_points = np.array(x_test_points)
y_true = np.sin(x_test_points[:,0:1] + x_test_points[:,1:2]) * x_test_points[:,2:3]

In [36]:
#looking at predicted and true solution side by side.

In [37]:
np.concatenate([y_predicted, y_true], axis=1)

array([[-0.1297535 , -0.1168255 ],
       [ 0.70116615,  0.72358866]])

Note that we need more training for further improving the accuracy.

### Saving the model to a specified location.

In [48]:
path_to_save_model = "saved_model/model_name"
diffusion_model.save_model(path_to_save_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/model_name\assets


### Loading the saved model 

In [6]:
path_to_load_model = "saved_model/model_name"

In [7]:
loaded_diffusion_model = fluidlearn.Solver()

In [8]:
loaded_diffusion_model(space_dim=2,
    time_dep=True,
    load_model=True,
    model_path=path_to_load_model)

### Predicting using loaded model

In [9]:
y_predicted = loaded_diffusion_model.predict(X_data)

In [10]:
y_predicted

array([[-0.10157388],
       [-0.4190994 ],
       [-0.7965628 ],
       ...,
       [-0.10375804],
       [ 0.05802408],
       [-0.00470909]], dtype=float32)